# RunnableSequence

- Author: [Jinu Cho](https://github.com/jinucho), [Lee Jungbin](https://github.com/leebeanbin)
- Peer Review: [Teddy Lee](https://github.com/teddylee777), [김무상](https://github.com/musangk), [전창원](https://github.com/changwonjeon)
- Proofread:
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/05-Memory/06-ConversationSummaryMemory.ipynb) [![Open in GitHub](https://img.shields.io/badge/Open%20in%20GitHub-181717?style=flat-square&logo=github&logoColor=white)](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/05-Memory/06-ConversationSummaryMemory.ipynb)

## Overview

This tutorial introduces three key tools in LangChain: `RunnableSequence`, `RunnableBranch`, and `RunnableLambda`, essential for building efficient and powerful AI applications.

`RunnableSequence` is a fundamental component that enables sequential processing pipelines, allowing structured and efficient handling of AI-related tasks. It provides automatic data flow management, error handling, and seamless integration with other LangChain components.

`RunnableBranch` enables structured decision-making by routing input through predefined conditions, simplifying complex branching scenarios.

`RunnableLambda` offers a flexible, function-based approach, ideal for lightweight transformations and inline processing.

**Key Features of these components:**

- **`RunnableSequence`:**
  - Sequential processing pipeline creation
  - Automatic data flow management
  - Error handling and monitoring
  - Support for async operations  

- **`RunnableBranch`:**
  - Dynamic routing based on conditions
  - Structured decision trees
  - Complex branching logic

- **`RunnableLambda`:**
  - Lightweight transformations
  - Function-based processing
  - Inline data manipulation

### Table of Contents

- [Overview](#overview)
- [Environment Setup](#environment-setup)
- [What is the RunnableSequence](#what-is-the-runnablesequence)
- [What is the RunnableBranch](#what-is-the-runnablebranch)
- [RunnableLambda](#runnablelambda)
- [RunnableBranch](#runnablebranch)
- [Comparison of RunnableBranch and RunnableLambda](#comparison-of-runnablebranch-and-runnablelambda)

### References
- [RunnableSequence API Reference](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.RunnableSequence.html)
- [LangChain Expression Language (LCEL)](https://python.langchain.com/docs/expression_language/interface)
- [RunnableBranch API Reference](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.branch.RunnableBranch.html)  
- [RunnableLambda API Reference](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.base.RunnableLambda.html)  
---

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

[Note]
- `langchain-opentutorial` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can check out the [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [1]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [2]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_core",
        "langchain_openai",
        "pydantic",
    ],
    verbose=False,
    upgrade=True,
)

You can alternatively set `OPENAI_API_KEY` in `.env` file and load it. 

[Note] This is not necessary if you've already set `OPENAI_API_KEY` in previous steps.

In [3]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "04-Routing",
    }
)

Environment variables have been set successfully.


In [3]:
# Load environment variables
# Reload any variables that need to be overwritten from the previous cell

from dotenv import load_dotenv

load_dotenv(override=True)

True

## What is the RunnableSequence

`RunnableSequence` is a fundamental component in LangChain that enables the creation of sequential processing pipelines. It allows developers to chain multiple operations together where the output of one step becomes the input of the next step.

### Key Concepts

1. **Sequential Processing**
   - Ordered execution of operations
   - Automatic data flow between steps
   - Clear pipeline structure

2. **Data Transformation**
   - Input preprocessing
   - State management
   - Output formatting

3. **Error Handling**
   - Pipeline-level error management
   - Step-specific error recovery
   - Fallback mechanisms

Let's explore these concepts with practical examples.

### Simple Example

First, we will create a Chain that classifies incoming questions into one of three categories: math, science, or other.

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

load_dotenv()

# Basic Example: Text Processing Pipeline
basic_chain = (
    # Step 1: Input handling and prompt creation
    PromptTemplate.from_template("Summarize this text in three sentences: {text}")
    # Step 2: LLM processing
    | ChatOpenAI(temperature=0)
    # Step 3: Output parsing
    | StrOutputParser()
)

# Example usage
result = basic_chain.invoke({"text": "This is a sample text to process."})
print(result)

This text is a sample for processing purposes. It is likely being used to test a system or program. The content is brief and straightforward.


### Basic Pipeline Creation

In this section, we'll explore how to create fundamental pipelines using RunnableSequence. We'll start with a simple text generation pipeline and gradually build more complex functionality.

**Understanding Basic Pipeline Structure**  
- Sequential Processing: How data flows through the pipeline
- Component Integration: Combining different LangChain components
- Data Transformation: Managing input/output between steps

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

"""
Basic Text Generation Pipeline
This demonstrates the fundamental way to chain components in RunnableSequence.

Flow:
1. PromptTemplate -> Creates the prompt with specific instructions
2. ChatOpenAI -> Processes the prompt and generates content
3. StrOutputParser -> Cleans and formats the output
"""

# Step 1: Define the basic text generation chain
basic_generation_chain = (
    # Create prompt template for AI content generation
        PromptTemplate.from_template(
            """Generate a detailed technical explanation about {topic} in AI/ML field.
            Include:
            - Core technical concepts
            - Implementation details
            - Real-world applications
            - Technical challenges
            """
        )
        # Process with LLM
        | ChatOpenAI(temperature=0.7)
        # Convert output to clean string
        | StrOutputParser()
)

# Example usage
basic_result = basic_generation_chain.invoke({"topic": "Transformer architecture in LLMs"})
print("Generated Content:", result)

Generated Content: This text is a sample for processing purposes. It is likely being used to test a system or program. The content is brief and straightforward.


### Advanced Analysis Pipeline


Building upon our basic pipeline, we'll now create a more sophisticated analysis system that processes and evaluates the generated content.

**Key Features**
- State Management: Maintaining context throughout the pipeline
- Structured Analysis: Organizing output in a clear format
- Error Handling: Basic error management implementation

In [10]:
from langchain_core.runnables import RunnableSequence, RunnablePassthrough, RunnableLambda
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
import time

# Step 1: Define the analysis prompt template
analysis_prompt = PromptTemplate.from_template(
    """Analyze this technical content and extract the most crucial insights:
    
    {generated_basic_content}
    
    Provide a concise analysis focusing only on the most important aspects:
    (Importance : You should use Notion Syntax and try highliting with underlines, bold, emoji for title or something you describe context)
    
    Output format markdown outlet:
    # Key Technical Analysis
    
    ## Core Concept Summary
    [Extract and explain the 2-3 most fundamental concepts]
    
    ## Critical Implementation Insights
    [Focus on crucial implementation details that make this technology work]
    
    ## Key Challenges & Solutions
    [Identify the most significant challenges and their potential solutions]
    """
)

# Step 2: Define the critical analysis chain
analysis_chain = RunnableSequence(
    first=analysis_prompt,
    middle=[ChatOpenAI(temperature=0)],
    last=StrOutputParser()
)

# Step 3: Define the basic generation chain
generation_prompt = RunnableLambda(lambda x: f"""Generate technical content about: {x['topic']}""")

basic_generation_chain = RunnableSequence(
    first=RunnablePassthrough(),
    middle=[generation_prompt],
    last=ChatOpenAI(temperature=0.7)
)

# Step 4: Define the state initialization function
def init_state(x):
    return {
        "topic": x["topic"],
        "start_time": time.strftime('%Y-%m-%d %H:%M:%S')
    }

init_step = RunnableLambda(init_state)

# Step 5: Define the content generation function
def generated_basic_content(x):
    content = basic_generation_chain.invoke({"topic": x["topic"]})
    return {
        **x,
        # "generated_basic_content": content.content
        # To create a comprehensive wrap-up, you can combine the previous basic result with new annotated analysis.
        "generated_basic_content": basic_result
    }

generate_step = RunnableLambda(generated_basic_content)

# Step 6: Define the analysis function
def perform_analysis(x):
    analysis = analysis_chain.invoke({"generated_basic_content": x["generated_basic_content"]})
    return {
        **x,
        "key_insights": analysis
    }

analysis_step = RunnableLambda(perform_analysis)

# Step 7: Define the output formatting function
def format_output(x):
    return {
        "timestamp": x["start_time"],
        "topic": x["topic"],
        "content": x["generated_basic_content"],
        "analysis": x["key_insights"],
        "formatted_output": f"""
# Technical Analysis Summary
Generated: {x['start_time']}

## Original Technical Content
{x['generated_basic_content']}

---

{x['key_insights']}
"""
    }

format_step = RunnableLambda(format_output)

# Step 8: Create the complete analysis pipeline
analysis_pipeline = RunnableSequence(
    first=init_step,
    middle=[
        generate_step,
        analysis_step
    ],
    last=format_step
)

<p align="left">
 <img src = "/Users/leejungbin/Downloads/LangChain-OpenTutorial/13-LangChain-Expression-Language/img/Runnable_Pipeline.png">
</p>

In [11]:
# Example usage
def run_analysis(topic: str):
    result = analysis_pipeline.invoke({"topic": topic})

    print("Analysis Timestamp:", result["timestamp"])
    print("\nTopic:", result["topic"])
    print("\nFormatted Output:", result["formatted_output"])

if __name__ == "__main__":
    run_analysis("Transformer attention mechanisms")

Analysis Timestamp: 2025-01-15 23:38:46

Topic: Transformer attention mechanisms

Formatted Output: 
# Technical Analysis Summary
Generated: 2025-01-15 23:38:46

## Original Technical Content
Transformer architecture in Large Language Models (LLMs) is a key advancement in the field of Artificial Intelligence (AI) and Machine Learning (ML). The core technical concept of the Transformer architecture lies in its ability to process sequential data, such as text, with parallelization and attention mechanisms. This architecture eliminates the need for recurrent neural networks (RNNs) and allows for more efficient training of large-scale language models.

At the heart of the Transformer architecture are self-attention mechanisms, which allow the model to weigh the importance of different input tokens when making predictions. This attention mechanism enables the model to capture long-range dependencies in the input sequence, making it well-suited for language modeling tasks. The Transformer ar

### Structured Evaluation Pipeline

In this section, we'll add structured evaluation capabilities to our pipeline, including proper error handling and validation.

**Features**
- Structured Output: Using schema-based parsing
- Validation: Input and output validation
- Error Management: Comprehensive error handling

In [13]:
"""
Structured Evaluation Pipeline

This demonstrates:
1. Custom output parsing with schema validation
2. Error handling at each pipeline stage
3. Comprehensive validation system
"""
from langchain_core.runnables import RunnableSequence, RunnablePassthrough, RunnableLambda
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_openai import ChatOpenAI
import json
import time

# Step 1: Define structured output schema
response_schemas = [
    ResponseSchema(
        name="technical_evaluation",
        description="Technical evaluation of the content",
        type="object",
        properties={
            "core_concepts": {
                "type": "array",
                "description": "Key technical concepts identified"
            },
            "implementation_details": {
                "type": "object",
                "properties": {
                    "complexity": {"type": "string"},
                    "requirements": {"type": "array"},
                    "challenges": {"type": "array"}
                }
            },
            "quality_metrics": {
                "type": "object",
                "properties": {
                    "technical_accuracy": {"type": "number"},
                    "completeness": {"type": "number"},
                    "clarity": {"type": "number"}
                }
            }
        }
    )
]

evaluation_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# Step 2: Create basic generation chain
generation_prompt = RunnableLambda(lambda x: f"""Generate technical content about: {x['topic']}""")
basic_generation_chain = RunnableSequence(
    first=RunnablePassthrough(),
    middle=[generation_prompt],
    last=ChatOpenAI(temperature=0.7)
)

# Step 3: Create analysis chain
analysis_prompt = RunnableLambda(lambda x: f"""Analyze the following content: {x['generated_content']}""")
analysis_chain = RunnableSequence(
    first=RunnablePassthrough(),
    middle=[analysis_prompt],
    last=ChatOpenAI(temperature=0)
)

# Step 4: Create evaluation chain
evaluation_prompt = RunnableLambda(
    lambda x: f"""
    Evaluate the following AI technical content:
    {x['generated_content']}
    
    Provide a structured evaluation following these criteria:
    1. Identify and list core technical concepts
    2. Assess implementation details
    3. Rate quality metrics (1-10)
    
    {evaluation_parser.get_format_instructions()}
    """
)

evaluation_chain = RunnableSequence(
    first=RunnablePassthrough(),
    middle=[evaluation_prompt, ChatOpenAI(temperature=0)],
    last=evaluation_parser
)

# Helper function for error handling
def try_or_error(func, error_list):
    try:
        return func()
    except Exception as e:
        error_list.append(str(e))
        return None

# Step 5: Create pipeline components
def init_state(x):
    return {
        "topic": x["topic"],
        "errors": [],
        "start_time": time.time()
    }

def generate_content(x):
    return {
        **x,
        "generated_content": try_or_error(
            lambda: basic_generation_chain.invoke({"topic": x["topic"]}).content,
            x["errors"]
        )
    }

def perform_analysis(x):
    return {
        **x,
        "analysis": try_or_error(
            lambda: analysis_chain.invoke({"generated_content": x["generated_content"]}).content,
            x["errors"]
        )
    }

def perform_evaluation(x):
    return {
        **x,
        "evaluation": try_or_error(
            lambda: evaluation_chain.invoke(x),
            x["errors"]
        ) if not x["errors"] else None
    }

def finalize_output(x):
    return {
        **x,
        "completion_time": time.time() - x["start_time"],
        "status": "success" if not x["errors"] else "error"
    }

# Step 6: Create integrated pipeline
def create_evaluation_pipeline():
    return RunnableSequence(
        first=RunnableLambda(init_state),
        middle=[
            RunnableLambda(generate_content),
            RunnableLambda(perform_analysis),
            RunnableLambda(perform_evaluation)
        ],
        last=RunnableLambda(finalize_output)
    )

# Example usage
def demonstrate_evaluation():
    pipeline = create_evaluation_pipeline()
    result = pipeline.invoke({"topic": "Transformer attention mechanisms"})

    print("Pipeline Status:", result["status"])
    if result["status"] == "success":
        print("\nEvaluation Results:", json.dumps(result["evaluation"], indent=2))
    else:
        print("\nErrors Encountered:", result["errors"])

    print(f"\nProcessing Time: {result['completion_time']:.2f} seconds")

if __name__ == "__main__":
    demonstrate_evaluation()

Pipeline Status: success

Evaluation Results: {
  "technical_evaluation": {
    "core_technical_concepts": [
      "Transformer model architecture",
      "Attention mechanisms",
      "Self-attention mechanisms",
      "Long-range dependencies",
      "Variable-length input sequences"
    ],
    "implementation_details": "The content provides a clear explanation of how attention mechanisms work in the Transformer model, specifically focusing on self-attention mechanisms and their advantages over traditional recurrent neural networks. It also highlights the ability of the Transformer model to handle variable-length input sequences efficiently.",
    "quality_metrics": {
      "clarity": 9,
      "depth": 8,
      "relevance": 10,
      "accuracy": 9
    }
  }
}

Processing Time: 8.35 seconds


## What is the RunnableBranch

`RunnableBranch` is a powerful tool that allows dynamic routing of logic based on input. It enables developers to flexibly define different processing paths depending on the characteristics of the input data.  

`RunnableBranch` helps implement complex decision trees in a simple and intuitive way. This greatly improves code readability and maintainability while promoting logic modularization and reusability.  

Additionally, `RunnableBranch` can dynamically evaluate branching conditions at runtime and select the appropriate processing routine, enhancing the system's adaptability and scalability.  

Due to these features, `RunnableBranch` can be applied across various domains and is particularly useful for developing applications with high input data variability and volatility.

By effectively utilizing `RunnableBranch`, developers can reduce code complexity and improve system flexibility and performance.

### Dynamic Logic Routing Based on Input

This section covers how to perform routing in LangChain Expression Language.

Routing allows you to create non-deterministic chains where the output of a previous step defines the next step. This helps bring structure and consistency to interactions with LLMs.

There are two primary methods for performing routing:

1. Returning a Conditionally Executable Object from `RunnableLambda` (*Recommended*)  
2. Using `RunnableBranch`

Both methods can be explained using a two-step sequence, where the first step classifies the input question as related to math, science, or other, and the second step routes it to the corresponding prompt chain.

### Simple Example

First, we will create a Chain that classifies incoming questions into one of three categories: math, science, or other.

In [14]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """Classify the given user question into one of `math`, `science`, or `other`. Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""
)

# Create the chain.
chain = (
    prompt
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()  # Use a string output parser.
)

Use the created chain to classify the question.

In [15]:
# Invoke the chain with a question.
chain.invoke({"question": "What is 2+2?"})

'math'

In [16]:
# Invoke the chain with a question.
chain.invoke({"question": "What is the law of action and reaction?"})

'science'

In [17]:
# Invoke the chain with a question.
chain.invoke({"question": "What is LangChain?"})

'other'

## RunnableLambda  

`RunnableLambda` is a type of `Runnable` designed to simplify the execution of a single transformation or operation using a lambda (anonymous) function. 

It is primarily used for lightweight, stateless operations where defining an entire custom `Runnable` class would be overkill.  

Unlike `RunnableBranch`, which focuses on conditional branching logic, `RunnableLambda` excels in straightforward data transformations or function applications.

Syntax  
- `RunnableLambda` is initialized with a single lambda function or callable object.  
- When invoked, the input value is passed directly to the lambda function.  
- The lambda function processes the input and returns the result.  

Now, let's create three sub-chains.

In [18]:
math_chain = (
    PromptTemplate.from_template(
        """You are an expert in math. \
Always answer questions starting with "Pythagoras once said...". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOpenAI(model="gpt-4o-mini")
)

science_chain = (
    PromptTemplate.from_template(
        """You are an expert in science. \
Always answer questions starting with "Isaac Newton once said...". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    | ChatOpenAI(model="gpt-4o-mini")
)

general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question concisely:

Question: {question}
Answer:"""
    )
    | ChatOpenAI(model="gpt-4o-mini")
)

### Using Custom Functions  

This is the recommended approach in the official LangChain documentation. You can wrap custom functions with `RunnableLambda` to handle routing between different outputs.

In [19]:
# Return each chain based on the contents included in the topic.


def route(info):
    if "math" in info["topic"].lower():
        return math_chain
    elif "science" in info["topic"].lower():
        return science_chain
    else:
        return general_chain

In [20]:
from operator import itemgetter
from langchain_core.runnables import RunnableLambda

full_chain = (
    {"topic": chain, "question": itemgetter("question")}
    | RunnableLambda(
        # Pass the routing function as an argument.
        route
    )
    | StrOutputParser()
)

In [21]:
# Invoke the chain with a math-related question.
full_chain.invoke({"question": "Please explain the concept of calculus."})

'Pythagoras once said, "Mathematics is the language in which God has written the universe." In the realm of mathematics, calculus is a branch that allows us to understand change and motion. It consists primarily of two fundamental concepts: differentiation and integration.\n\nDifferentiation is concerned with how a function changes at any given point, essentially measuring the rate of change or the slope of a curve. This helps in understanding how quantities vary with respect to one another.\n\nOn the other hand, integration focuses on accumulating quantities, such as finding the area under a curve. It can be thought of as the reverse process of differentiation.\n\nTogether, these concepts enable us to solve complex problems related to motion, area, and other dynamic systems. Calculus serves as a foundational tool in fields ranging from physics to economics, allowing us to model and analyze real-world phenomena.'

In [22]:
# Invoke the chain with a science-related question.
full_chain.invoke({"question": "How is gravitational acceleration calculated?"})

"Isaac Newton once said that gravity is a force that attracts two bodies toward each other, and he formulated the law of universal gravitation. Gravitational acceleration, specifically near the surface of the Earth, can be calculated using the equation \\( g = \\frac{GM}{r^2} \\), where \\( g \\) is the acceleration due to gravity, \\( G \\) is the universal gravitational constant, \\( M \\) is the mass of the Earth, and \\( r \\) is the distance from the center of the Earth to the point of measurement. Near the Earth's surface, this value is approximately \\( 9.81 \\, \\text{m/s}^2 \\)."

In [23]:
# Invoke the chain with a general question.
full_chain.invoke({"question": "What is RAG (Retrieval Augmented Generation)?"})

'RAG (Retrieval-Augmented Generation) is a framework that combines retrieval and generation techniques in natural language processing. It retrieves relevant information from a knowledge base and uses it to enhance the generation of responses, thereby improving the quality and relevance of the output. This approach is particularly useful in tasks like question answering and dialogue generation, where context and accurate information are critical.'

## RunnableBranch

`RunnableBranch` is a special type of `Runnable` that allows you to define conditions and corresponding Runnable objects based on input values.

However, it does not provide functionality that cannot be achieved with custom functions, so using custom functions is generally recommended.

Syntax

- `RunnableBranch` is initialized with a list of (condition, Runnable) pairs and a default Runnable.
- When invoked, the input value is passed to each condition sequentially.
- The first condition that evaluates to True is selected, and the corresponding Runnable is executed with the input value.
- If no condition matches, the `default Runnable` is executed.

In [25]:
from operator import itemgetter
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    # Check if the topic contains "math" and execute math_chain if true.
    (lambda x: "math" in x["topic"].lower(), math_chain),
    # Check if the topic contains "science" and execute science_chain if true.
    (lambda x: "science" in x["topic"].lower(), science_chain),
    # If none of the above conditions match, execute general_chain.
    general_chain,
)

# Define the full chain that takes a topic and question, routes it, and parses the output.
full_chain = (
    {"topic": chain, "question": itemgetter("question")} | branch | StrOutputParser()
)

Execute the full chain with each question.

In [26]:
full_chain.invoke({"question": "Please explain the concept of calculus."})

"Pythagoras once said that the pursuit of knowledge is essential for understanding the world around us. Calculus is a branch of mathematics that focuses on how things change. It provides tools for modeling and analyzing dynamic systems and is primarily divided into two main concepts: differentiation and integration.\n\nDifferentiation is about finding the rate at which a quantity changes. For instance, if you have a curve representing the position of an object over time, the derivative of that curve gives you the object's velocity at any instant.\n\nIntegration, on the other hand, deals with accumulation. It allows us to calculate areas under curves, total quantities, or the sum of infinitely many infinitesimal parts. In essence, while differentiation breaks down a function to understand its behavior at specific points, integration builds up from those points to find total values.\n\nTogether, these concepts offer profound insights into not just mathematics, but also physics, engineeri

In [27]:
full_chain.invoke({"question": "How is gravitational acceleration calculated?"})

'Isaac Newton once said, "What goes up must come down." To calculate gravitational acceleration, we use the formula \\( g = \\frac{G \\cdot M}{r^2} \\), where \\( g \\) is the acceleration due to gravity, \\( G \\) is the universal gravitational constant approximately equal to \\( 6.674 \\times 10^{-11} \\, \\text{N(m/kg)}^2 \\), \\( M \\) is the mass of the object attracting another object (like the Earth), and \\( r \\) is the distance from the center of mass of the attracting object to the center of the object being attracted (like an object on the surface of the Earth). On the surface of the Earth, this results in an approximate gravitational acceleration of \\( 9.81 \\, \\text{m/s}^2 \\).'

In [28]:
full_chain.invoke({"question": "What is RAG (Retrieval Augmented Generation)?"})

'RAG (Retrieval Augmented Generation) is a machine learning approach that enhances natural language generation by integrating external information retrieval. It combines a generative model with a retrieval mechanism, allowing the system to fetch relevant documents or data from a knowledge base to improve the accuracy and relevance of the generated text. This method is particularly useful in generating responses that require factual knowledge or specific information.'

## Building an AI Learning Assistant

Let's apply what we've learned about Runnable components to build a practical AI Learning Assistant. This system will help students by providing tailored responses based on their questions.

First, let's set up our core components:

In [40]:
from langchain_core.runnables import RunnableSequence, RunnableBranch, RunnableLambda
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
import json

# Question Classification Component
question_classifier = RunnableSequence(
    first=PromptTemplate.from_template(
        """Classify this question into one of: beginner, intermediate, advanced
        Consider:
        - Complexity of concepts
        - Prior knowledge required
        - Technical depth needed
        
        Question: {question}
        
        Return only the classification word in lowercase."""
    ),
    middle=[ChatOpenAI(temperature=0)],
    last=StrOutputParser()
)

# Example Generator Component
example_generator = RunnableSequence(
    first=PromptTemplate.from_template(
        """Generate a practical example for this concept.
        Level: {level}
        Question: {question}
        
        If code is needed, provide it in appropriate markdown format."""
    ),
    middle=[ChatOpenAI(temperature=0.7)],
    last=StrOutputParser()
)

Next, let's create our response generation strategy:

In [41]:
# Response Generation Strategy
response_strategy = RunnableBranch(
    (
        lambda x: x["level"] == "beginner",
        RunnableSequence(
            first=PromptTemplate.from_template(
                """Explain in simple terms for a beginner:
                Question: {question}
                
                Use simple analogies and avoid technical jargon."""
            ),
            middle=[ChatOpenAI(temperature=0.3)],
            last=StrOutputParser()
        )
    ),
    (
        lambda x: x["level"] == "intermediate",
        RunnableSequence(
            first=PromptTemplate.from_template(
                """Provide a detailed explanation with practical examples:
                Question: {question}
                
                Include relevant technical concepts and use cases."""
            ),
            middle=[ChatOpenAI(temperature=0.3)],
            last=StrOutputParser()
        )
    ),
    # Default case (advanced)
    RunnableSequence(
        first=PromptTemplate.from_template(
            """Give an in-depth technical explanation:
            Question: {question}
            
            Include advanced concepts and detailed technical information."""
        ),
        middle=[ChatOpenAI(temperature=0.3)],
        last=StrOutputParser()
    )
)

Now, let's create our main pipeline:

In [42]:
def format_response(x):
    return {
        "question": x["question"],
        "level": x["level"],
        "explanation": x["response"],
        "example": x["example"],
        "metadata": {
            "difficulty": x["level"],
            "timestamp": datetime.now().isoformat()
        }
    }

# Main Learning Assistant Pipeline
learning_assistant = RunnableSequence(
    first=RunnableLambda(lambda x: {"question": x["question"]}),
    middle=[
        RunnableLambda(lambda x: {
            **x,
            "level": question_classifier.invoke({"question": x["question"]})
        }),
        RunnableLambda(lambda x: {
            **x,
            "response": response_strategy.invoke(x),
            "example": example_generator.invoke(x)
        })
    ],
    last=RunnableLambda(format_response)
)

Let's try out our assistant:

In [43]:
async def run_assistant():
    # Example questions for different levels
    questions = [
        "What is a variable in Python?",
        "How does dependency injection work?",
        "Explain quantum computing qubits"
    ]
    
    for question in questions:
        result = await learning_assistant.ainvoke({"question": question})
        print(f"\nQuestion: {result['question']}")
        print(f"Difficulty Level: {result['level']}")
        print(f"\nExplanation: {result['explanation']}")
        print(f"\nExample: {result['example']}")
        print("\n" + "="*50)

# For Jupyter environments
import nest_asyncio
nest_asyncio.apply()

# Run the assistant
if __name__ == "__main__":
    asyncio.run(run_assistant())


Question: What is a variable in Python?
Difficulty Level: beginner

Explanation: In Python, a variable is like a container that holds information. Just like a box can hold different things like toys or clothes, a variable can hold different types of data like numbers or words. You can give a variable a name so you can easily refer to it later in your code. Think of it as a label on a box that helps you find what you're looking for.

Example: A variable in Python is a way to store and manipulate data within a program. It is essentially a placeholder that can hold different values.

Example:
```python
# Define a variable called 'name' and assign it the value "Alice"
name = "Alice"

# Print out the value stored in the variable 'name'
print(name)
```

In this example, the variable `name` is storing the string "Alice". The `print` statement then outputs the value of the variable, which is "Alice".


Question: How does dependency injection work?
Difficulty Level: intermediate

Explanation: 

## Comparison of RunnableSequence, RunnableBranch, and RunnableLambda

| Criteria | RunnableSequence | RunnableBranch | RunnableLambda |
|----------|------------------|----------------|----------------|
| Primary Purpose | Sequential pipeline processing | Conditional routing and branching | Simple transformations and functions |
| Condition Definition | No conditions, sequential flow | Each condition defined as `(condition, runnable)` pair | All conditions within single function (`route`) |
| Structure | Linear chain of operations | Tree-like branching structure | Function-based transformation |
| Readability | Very clear for sequential processes | Becomes clearer as conditions increase | Very clear for simple logic |
| Maintainability | Easy to maintain step-by-step flow | Clear separation between conditions and runnables | Can become complex if function grows large |
| Flexibility | Flexible for linear processes | Must follow `(condition, runnable)` pattern | Allows flexible condition writing |
| Scalability | Add or modify pipeline steps | Requires adding new conditions and runnables | Expandable by modifying function |
| Error Handling | Pipeline-level error management | Branch-specific error handling | Basic error handling |
| State Management | Maintains state throughout pipeline | State managed per branch | Typically stateless |
| Recommended Use Case | When you need ordered processing steps | When there are many conditions or maintainability is priority | When conditions are simple or function-based |
| Complexity Level | Medium to High | Medium | Low |
| Async Support | Full async support | Limited async support | Basic async support |